In [7]:
import eikon as ek
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
ek.set_app_key('b499c15480534749a47879878c132348412d0c30')

getting the sp500 stock names from wikipedia 

In [9]:
data, _ = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

In [28]:
symbols = data['Symbol'].tolist()
symbols[:10]

['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES']

the symbols come in the <span style="color:red">'ticker'</span> form, need to convert them to <span style="color:red">'RIC'</span> or otherwise wont be able to retrieve any data

In [333]:
def ticker_to_rics(symbols: list) -> list: 
    rics = ek.symbology.get_symbology(symbols, 
                                      from_symbol_type='ticker',
                                      to_symbol_type='RIC',
                                      bestMatch=False)
    rics = rics['RICs'].apply(lambda x: x[0]).values
    return rics.tolist()

retrieving the data for all stocks of SP500 takes about 30 seconds 

In [338]:
df, _ =  ek.get_data(instruments=ticker_to_rics(symbols), 
                     fields=['TR.ClosePrice', 'TR.PE', 'EARNINGS'])

In [356]:
df.head(10)

,Instrument,Close Price,P/E (Daily Time Series Ratio),EARNINGS
0,MMM,152.12,17.244956,8.8211
1,ABT,101.53,58.730976,1.7287
2,ABBV.K,96.04,16.994079,5.6514
3,ABMD.O,298.91,67.544375,4.4282
4,ACN,223.98,29.176599,7.6744
5,ATVI.O,81.70,40.172166,2.0213
6,ADBE.O,438.88,57.483152,7.5901
7,AMD.O,78.20,187.045231,0.5032
8,AAP,152.87,27.860143,5.4936
9,AES,15.42,35.248937,0.4375


some of the values for P/E will be NaN for the symbols with negative earnings, thats fine

In [393]:
df[df.isna().any(axis=1) == True].head(10)

,Instrument,Close Price,P/E (Daily Time Series Ratio),EARNINGS
24,MO,41.580,NaN,-0.4290
28,AAL.O,11.180,NaN,-8.1735
39,.ADI,4304.736,NaN,NaN
44,APA.O,16.010,NaN,-21.2178
60,BKR,15.330,NaN,-15.7680
71,BA,161.950,NaN,-5.0347
75,BIFc1,NaN,NaN,NaN
85,CAH,56.480,NaN,-14.0385
87,CCL,14.340,NaN,-4.0167
95,CNP,19.130,NaN,-1.0983


#### TODO address symbols with nan earnings below

it looks like those symbols got changed to wrong rics 

In [391]:
nan[nan['EARNINGS'].isnull()]

,Instrument,Close Price,P/E (Daily Time Series Ratio),EARNINGS
39,.ADI,4304.736,NaN,NaN
75,BIFc1,NaN,NaN,NaN
117,CLc1,40.330,NaN,NaN
178,EScv1,3267.750,NaN,NaN
437,AEXc1,550.200,NaN,NaN
